# Classificações do modelo

## IMPORTS

In [1]:
import re
from string import punctuation
import joblib
import nltk
from string import punctuation
from unidecode import unidecode
import unidecode
from sklearn.feature_extraction.text import TfidfVectorizer
import pickle

## Carregando o modelo

In [2]:
modelo = joblib.load('./model/nb_predictor.joblib');
with open('./model/tfidf.pkl', 'rb') as file:
    feature_vocabulary = pickle.load(file);

In [3]:
# Load the feature vocabulary from feature.pkl
with open('./model/tfidf.pkl', 'rb') as file:
    feature_vocabulary = pickle.load(file)

# Now, feature_vocabulary contains the vocabulary from feature.pkl
tfidf = TfidfVectorizer()
vidf = tfidf.fit_transform(feature_vocabulary)

## FUNÇÕES

### Porcessar texto

In [4]:
def processar_texto(texto):
    stemmer = nltk.RSLPStemmer()
    stopwords = nltk.corpus.stopwords.words('portuguese')
    texto = re.sub(r'@[A-Za-z0-9]+', '', texto) # Removendo menções
    texto = re.sub(r'#\w+', '', texto) # Removendo #hashtags
    texto = re.sub(r'#', '', texto)
    texto = re.sub(r'_', '', texto)
    texto = re.sub(r'RT[\s]+', '', texto)
    texto = re.sub(r'https?:\/\/\S+', '', texto)
    texto = re.sub(r'\n', '', texto)
    pontuacao = []
    for ponto in punctuation:
        pontuacao.append(ponto)
    # Remove caracteres especiais
    texto = re.sub(r'[^\w\s]', '', texto)
    # Converte para minúsculas
    texto = texto.lower()
    texto = [stemmer.stem(palavra) for palavra in texto.split() if palavra not in stopwords]
    # Une acentos
    texto = [unidecode.unidecode(palavra) for palavra in texto]
    return ' '.join(texto)

### Predict it

In [5]:
def predict_it(frase):
    # Aplica a função de pré-processamento
    texto_processado = processar_texto(frase)
    # Transforma o texto em um vetor TF-IDF
    vetor_tfidf = tfidf.transform([texto_processado])
    # Prevê a classificação da frase
    previsao = modelo.predict(vetor_tfidf)
    probabilidades = modelo.predict_proba(vetor_tfidf).astype('float')[0]
    pos = probabilidades[1]
    neg = probabilidades[0]
    if pos < 0.51 and neg < 0.51:
        return 'Neutro'
    else:
        return previsao.astype('str')[0]

### Fazendo predições

In [6]:
predict_it('Eu odeio overfitting.')

np.str_('Negativo')

In [7]:
predict_it('Parabéns, você foi selecionado no nosso processo seletivo.')

np.str_('Positivo')